In [21]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import random_split, DataLoader

In [8]:
# Define the path to your dataset
data_dir = '/Users/harrynguyen/Documents/GitHub/APS360_Yoga_Poses_Classification/BaselineModel/skeletonized'
#careful

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256
    transforms.ToTensor(),          # Convert images to PyTorch tensors
])

# Load dataset
dataset = datasets.ImageFolder(data_dir, transform=transform)

In [44]:
import sys
!{sys.executable} -m pip install openpose

ERROR: Could not find a version that satisfies the requirement openpose (from versions: none)
ERROR: No matching distribution found for openpose


In [ ]:
from openpose import pyopenpose as op

params = dict()
params["model_folder"] = data_dir
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

def extract_keypoints(image):
    datum = op.Datum()
    datum.cvInputData = image
    opWrapper.emplaceAndPop([datum])
    keypoints = datum.poseKeypoints
    return keypoints


In [11]:
# Define the split sizes
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [20]:
len(dataset.classes)

87

In [22]:
# Define batch size
batch_size = 64

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
### Output = (N - F + 2P)/S + 1

In [38]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes = len(dataset.classes)):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 64 * 64, 512)  # Corrected input size
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 64 * 64)  # Corrected view size
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [24]:
def get_accuracy(model, dataloader):
  "Gets accuracy for final accuracy and accuracy plots"
  correct=0
  total=0
  with torch.no_grad():
    for imgs, labels in dataloader:

      #To Enable GPU Usage
      if use_cuda and torch.cuda.is_available():
          # print("consist")
          imgs = imgs.cuda()
          labels = labels.cuda()

      outputs = model(imgs)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      # print(predicted)#weird size mismatch
      # print(labels)

      correct += (predicted == labels).sum().item()#adds sum of them
  return correct/total

In [40]:
import time

def train_plot(model, num_epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  iters,losses,train_acc,valid_acc =[],[],[],[]

  start_time=time.time()
  for epoch in range(num_epochs):  # Loop over the dataset multiple times
    running_loss = 0.0
    model.train()  # Set the model to training mode

    for i, data in enumerate(train_loader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data


      #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          # model.cuda()
          # print("consist")
          inputs = inputs.cuda()
          labels = labels.cuda()
        #############################################

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            iters.append(epoch * 100 + i)
            losses.append(running_loss / 100)
            train_acc.append(get_accuracy(model, train_loader))
            valid_acc.append(get_accuracy(model, val_loader))

            print(f'Epoch {epoch + 1}, Minibatch {i + 1}, Loss: {running_loss / 100:.3f}')
            print("\tTime Elapsed: % 6.2f s " % (time.time()-start_time))
            running_loss = 0.0

#done training
  print(iters)
  print(losses)

  plt.title("Training Curve")
  plt.plot(iters, losses, label="Train")
  plt.xlabel("Iterations")
  plt.ylabel("Loss")
  plt.show()


  plt.title("Accuracy Train Curve")
  plt.plot(iters, valid_acc, label="Validation")
  plt.plot(iters, train_acc, label="Training")
  plt.legend(loc='best')
  plt.xlabel("Iterations")
  plt.ylabel("Accuracy")
  plt.show()
  print("Final Training Accuracy: {}".format(train_acc[-1]))
  print("Final Validation Accuracy: {}".format(valid_acc[-1]))



In [42]:
model = SimpleCNN()

use_cuda = True


if use_cuda and torch.cuda.is_available():
  model.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

train_plot(model, num_epochs=1)

CUDA is not available.  Training on CPU ...


KeyboardInterrupt: 

In [ ]:
test_acc=get_accuracy(model, test_loader)
print(f'Final Test Accuracy: {100 * test_acc:.2f}%')